In [ ]:
%load_ext autoreload
%autoreload 2

from torch.utils.data import DataLoader
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
import seaborn as sns
from models.transformer import Transformer
from train import Trainer
from dataloader import EnglishToSpanish

# Model

In [226]:
trainer = Trainer()
data = EnglishToSpanish()

Corpus eng: 8
Corpus esp: 7


In [227]:
tm = Transformer()

In [228]:
trainer.fit(tm, data)

0/6 | loss: 0.49309706687927246
1/6 | loss: 0.1395762711763382
2/6 | loss: 0.6052873730659485
3/6 | loss: 0.6344212889671326
4/6 | loss: 0.32110315561294556
5/6 | loss: 0.4618662893772125
EPOCH 1 train loss: 0.4425585741798083

0/6 | loss: 0.3318556845188141
1/6 | loss: 0.25888094305992126
2/6 | loss: 0.23945696651935577
3/6 | loss: 0.22817954421043396
4/6 | loss: 0.04012959823012352
5/6 | loss: 0.2955445647239685
EPOCH 2 train loss: 0.23234121687710285

0/6 | loss: 0.12120409309864044
1/6 | loss: 0.2609812021255493
2/6 | loss: 0.007978265173733234
3/6 | loss: 0.2977290451526642
4/6 | loss: 0.09162497520446777
5/6 | loss: 0.27540868520736694
EPOCH 3 train loss: 0.17582104432707032

0/6 | loss: 0.10438761860132217
1/6 | loss: 0.15139925479888916
2/6 | loss: 0.004972574766725302
3/6 | loss: 0.25964584946632385
4/6 | loss: 0.10972370952367783
5/6 | loss: 0.1954764425754547
EPOCH 4 train loss: 0.13760090828873217

0/6 | loss: 0.14001548290252686
1/6 | loss: 0.20068849623203278
2/6 | loss: 

([0.4425585741798083,
  0.23234121687710285,
  0.17582104432707032,
  0.13760090828873217,
  0.14413930118704835,
  0.1794399751815945,
  0.15416353583956757,
  0.09761236421763897,
  0.07594234480833013,
  0.07438698022936781,
  0.06340151893285413,
  0.06044377840589732,
  0.06153416847034047,
  0.047949080976347126,
  0.05981623303766052,
  0.05274396083162477,
  0.054020185101156436,
  0.06964714115019888,
  0.056300004032285265,
  0.05115451489109546],
 [])

In [ ]:
print(tm)

In [229]:
dl = iter(DataLoader(data))

# Inference

In [233]:
inp, out, inp_pad, out_pad, src, tgt = next(dl)

In [234]:
data._idx_to_token(inp, is_en=True)

[['how',
  'are',
  'you',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>']]

In [235]:
data._idx_to_token(out)

[['como',
  'estas',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>',
  '<PAD>']]

In [236]:
tm.eval()
sos = torch.repeat_interleave(data.sos, 1, 0)
out1 = torch.cat([sos], 1)
print(out1)

tensor([[0]])


In [262]:
R = tm(inp, out1, inp_pad, 0)
last_pred = R[0,-1]
print(f"Adding with prob. {torch.max(last_pred)}")
r = torch.argmax(last_pred)
out1 = torch.cat([out1, r.reshape(1, 1)], 1)
print(out1)

Adding with prob. 0.987286388874054
tensor([[0, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]])


In [263]:
data._idx_to_token(out1)

[['<SOS>',
  'como',
  'estas',
  'estas',
  'estas',
  'estas',
  'estas',
  'estas',
  'estas',
  'estas',
  'estas',
  'estas',
  'estas',
  'estas',
  'estas',
  'estas',
  'estas',
  'estas',
  'estas',
  'estas',
  'estas',
  'estas']]

# Testing

In [264]:
R = tm(inp, out, inp_pad, out_pad)
data._idx_to_token(torch.argmax(R, 2))

[['estas',
  'estas',
  '<EOS>',
  '<EOS>',
  '<EOS>',
  '<EOS>',
  '<EOS>',
  '<EOS>',
  '<EOS>',
  '<EOS>',
  '<EOS>',
  '<EOS>',
  '<EOS>',
  '<EOS>',
  '<EOS>',
  '<EOS>']]